In [1]:
import os
import struct
import numpy as np

In [27]:
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.svm import LinearSVC

In [3]:
fname_img = 'train-images_mnist.idx3-ubyte'
fashion_fname_img = 'train-images_fashion-idx3-ubyte'
fname_lbl = 'train-labels_mnist.idx1-ubyte'
fashion_fname_lbl = 'train-labels_fashion-idx1-ubyte'

In [4]:
with open(fname_lbl, 'rb') as flbl:
    magic, num = struct.unpack(">II", flbl.read(8))
    lbl = np.fromfile(flbl, dtype=np.int8)

with open(fname_img, 'rb') as fimg:
    magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

In [5]:

def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    image = np.reshape(image , [28,28])
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

In [6]:
import numpy as np
import keras 

Using TensorFlow backend.


In [7]:
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import  Dropout, Flatten

In [8]:
train_y  = keras.utils.np_utils.to_categorical(lbl ,  10  )
train_x = np.asarray(img)
train_y = np.asarray(train_y) 
print train_y.shape
print train_x.shape 

(60000, 10)
(60000, 28, 28)


In [9]:
train_x.shape

(60000, 28, 28)

In [10]:
train_x = np.reshape(train_x , [60000,28,28,1])

In [12]:
model = Sequential()
model.reset_states() 

 
model.add(Convolution2D(64, 3, 3,  input_shape = train_x[0].shape , name = "con1") ) 
model.add(Activation('relu' , name = "act1"))
model.add(MaxPooling2D(pool_size=(2, 2), name = "pool1") )

model.add(Convolution2D(32, 3, 3, name = "con2"))
model.add(Activation('relu', name = "act2" ))
model.add(MaxPooling2D(pool_size=(2, 2), name = "pool2"))

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))

In [13]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt,  metrics=['accuracy'])
#model.fit( train_x , train_y  , batch_size= 250 ,   nb_epoch = 5 ,  shuffle=True)

In [16]:
for ii, val  in enumerate(model.layers ): 
    print ii , val.name

0 con1
1 act1
2 pool1
3 con2
4 act2
5 pool2
6 flatten_2
7 dense_2
8 activation_2


In [14]:
model.load_weights("mnist_original_model.h5")

model.save_weights("mnist_original_model.h5")

In [15]:
from keras import backend as K
get_layer_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[5].output])
get_layer_output_early = K.function([model.layers[0].input, K.learning_phase()], [model.layers[2].output])

In [16]:
with open(fashion_fname_lbl , 'rb') as flbl:
    magic, num = struct.unpack(">II", flbl.read(8))
    fashion_lbl = np.fromfile(flbl, dtype=np.int8)

with open(fashion_fname_img, 'rb') as fimg:
    magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
    fashion_img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

yy = fashion_lbl
inds = yy.argsort()

fashion_img= fashion_img[inds]
fashion_lbl=  fashion_lbl[inds]

In [17]:
from IPython.core.display import display, HTML

In [18]:
display(HTML('<h1>NOW RUNNING EXPTS ON REDUCED FASHION MNIST DATA!</h1>'))
display(HTML('<h1>===============================================</h1>'))


In [19]:
train_fashion_x = []
train_fashion_y = []

test_fashion_x = []
test_fashion_y = []

take_number_of_training_images = 5000
for i in range(0,len(fashion_lbl),6000 ): 
    train_fashion_x.extend(fashion_img[i:i+take_number_of_training_images])
    train_fashion_y.extend(fashion_lbl[i:i+take_number_of_training_images])
    
    test_fashion_x.extend(fashion_img[i+take_number_of_training_images:i+6000])
    test_fashion_y.extend(fashion_lbl[i+take_number_of_training_images:i+6000])

train_fashion_x = np.asarray(train_fashion_x)
train_fashion_x = np.reshape(train_fashion_x , [len(train_fashion_x),28,28,1])
train_fashion_y  = np.asarray(train_fashion_y)

test_fashion_y  = np.asarray(test_fashion_y)
test_fashion_x = np.asarray(test_fashion_x)
test_fashion_x = np.reshape(test_fashion_x , [len(test_fashion_x),28,28,1])

In [20]:
print len(train_fashion_x)
print len(train_fashion_y)
print len(test_fashion_x)
print len(test_fashion_y)

50000
50000
10000
10000


In [21]:
import random
z = zip(test_fashion_x, test_fashion_y)
random.shuffle(z)
test_fashion_x, test_fashion_y = zip(*z)
test_fashion_x = np.asarray(test_fashion_x)
test_fashion_y = np.asarray(test_fashion_y)

In [22]:
def get_feats_fashion_mnist_training( inputs ): 
     
    layer_out = get_layer_output([inputs, 0 ])[0]  
    return layer_out.reshape([len(layer_out), layer_out[0].shape[0]*layer_out[0].shape[1]*layer_out[0].shape[2]  ] )
def get_early_feats_fashion_mnist_training( inputs ): 
    layer_out = get_layer_output_early([inputs, 0 ])[0] 
    return layer_out.reshape([len(layer_out), layer_out[0].shape[0]*layer_out[0].shape[1]*layer_out[0].shape[2]  ] )

In [23]:
def train_linear_model( train_x , train_y , test_x , test_y ) : 
    train_x = train_x.reshape( len(train_x), train_x[0].shape[0]*train_x[0].shape[0] )
    test_x = test_x.reshape( len(test_x), test_x[0].shape[0]*test_x[0].shape[0]  )
    
    dict = { }
     
    clf = linear_model.SGDClassifier()
    clf.fit(train_x, train_y  )
    print "model fitted"
    preds = clf.predict( test_x )
    acc =  accuracy_score(test_y , preds )
    dict["linear_SVM"] = acc
   
    return dict 
     
    clf = LinearSVC()
    clf.fit(train_x, train_y  )
    print "model fitted"
    preds = clf.predict( test_x )
    acc =  accuracy_score(test_y , preds )
    dict["linear_SVC"] = acc

 
    SVC_rbf = SVC( ) 
    SVC_rbf.fit(train_x, train_y  )
    print "model fitted"
    preds = SVC_rbf.predict( test_x ) 
    acc =  accuracy_score(test_y , preds )
    dict["RBF"] = acc
    return dict  

In [24]:
accuracy_scores = { }

In [ ]:
fashion_feats = get_early_feats_fashion_mnist_training(train_fashion_x)
fashion_feats = fashion_feats.reshape([len(fashion_feats), fashion_feats[0].shape[0]  ])
print fashion_feats.shape 

clf = LinearSVC()
clf.fit(fashion_feats, train_fashion_y  )
test_fashion_feats = get_early_feats_fashion_mnist_training(test_fashion_x[:10000] ) 
preds = clf.predict(test_fashion_feats)
from sklearn.metrics import accuracy_score
print accuracy_score(test_fashion_y[:10000], preds  )

accuracy_scores["early_feats_linear"] = accuracy_score(test_fashion_y[:10000], preds  )

In [ ]:
fashion_feats = get_feats_fashion_mnist_training(train_fashion_x)
fashion_feats = fashion_feats.reshape([len(fashion_feats), fashion_feats[0].shape[0]  ])
print fashion_feats.shape 

clf = LinearSVC()
clf.fit(fashion_feats, train_fashion_y  )
test_fashion_feats = get_feats_fashion_mnist_training(test_fashion_x[:10000] ) 
preds = clf.predict(test_fashion_feats)
from sklearn.metrics import accuracy_score
print accuracy_score(test_fashion_y[:10000], preds  )

accuracy_scores["late_feats_linear"] = accuracy_score(test_fashion_y[:10000], preds  )

In [28]:
accuracy_scores["linear_models"] = train_linear_model( train_fashion_x , train_fashion_y, test_fashion_x[:10000] , test_fashion_y[:10000] )

model fitted


In [29]:
accuracy_scores

{'linear_models': {'linear_SVM': 0.80220000000000002}}

In [30]:
fashion_model = Sequential()
fashion_model.reset_states() 
fashion_model.add(Convolution2D( 64  , 3, 3 , input_shape = train_x[0].shape   ))  
fashion_model.add(Activation('tanh'))
fashion_model.add(MaxPooling2D(pool_size=(3, 3)))
fashion_model.add(Flatten())
fashion_model.add(Dense(10))
fashion_model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
fashion_model.reset_states( )

train_fashion_y_categorical =  keras.utils.np_utils.to_categorical(train_fashion_y  ,  10  )
test_fashion_y_categorical =  keras.utils.np_utils.to_categorical(test_fashion_y  ,  10  )

fashion_model.compile(loss='categorical_crossentropy', optimizer=opt,  metrics=['accuracy'])
fashion_history = fashion_model.fit( train_fashion_x , train_fashion_y_categorical , batch_size= 20, nb_epoch = 25, validation_data=(test_fashion_x[:10000],test_fashion_y_categorical[:10000]) )
accuracy_scores["cnn_on_fashion_mnist"] = fashion_history.history["val_acc"][-1]

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 50s - loss: 0.4843 - acc: 0.8262 - val_loss: 0.3716 - val_acc: 0.8662
Epoch 2/25
50000/50000 [==============================] - 50s - loss: 0.3441 - acc: 0.8762 - val_loss: 0.3348 - val_acc: 0.8796
Epoch 3/25
50000/50000 [==============================] - 49s - loss: 0.3098 - acc: 0.8882 - val_loss: 0.3268 - val_acc: 0.8845
Epoch 4/25
50000/50000 [==============================] - 49s - loss: 0.2904 - acc: 0.8960 - val_loss: 0.3015 - val_acc: 0.8919
Epoch 5/25
50000/50000 [==============================] - 49s - loss: 0.2736 - acc: 0.9018 - val_loss: 0.2996 - val_acc: 0.8956
Epoch 6/25
50000/50000 [==============================] - 49s - loss: 0.2629 - acc: 0.9066 - val_loss: 0.3020 - val_acc: 0.8939
Epoch 7/25
50000/50000 [==============================] - 49s - loss: 0.2521 - acc: 0.9104 - val_loss: 0.2947 - val_acc: 0.8961
Epoch 8/25
50000/50000 [==============================

In [31]:
custom_model = Sequential()
custom_model.reset_states() 
custom_model.add(Convolution2D(64, 3, 3,  input_shape = train_x[0].shape, name = "con1" ))
custom_model.add(Activation('relu', name= "act1" ))
custom_model.add(MaxPooling2D(pool_size=(2, 2), name = "pool1" ))

custom_model.add(Convolution2D(32, 3, 3))
custom_model.add(Activation('relu'))
custom_model.add(MaxPooling2D(pool_size=(2, 2)))

custom_model.add(Flatten())
custom_model.add(Dense(10))
custom_model.add(Activation('softmax'))

custom_model.load_weights("mnist_original_model.h5",by_name=True)

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
custom_model.compile(loss='categorical_crossentropy', optimizer=opt,  metrics=['accuracy'])

train_fashion_y_categorical =  keras.utils.np_utils.to_categorical(train_fashion_y  ,  10  )
test_fashion_y_categorical  =  keras.utils.np_utils.to_categorical(test_fashion_y  ,  10  )


custom_history = custom_model.fit( train_fashion_x , train_fashion_y_categorical  , batch_size= 20 ,nb_epoch = 25 ,validation_data=(test_fashion_x[:10000],test_fashion_y_categorical[:10000]) )
accuracy_scores["custom_on_fashion_mnist"] = custom_history.history["val_acc"][-1]

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 97s - loss: 6.0471 - acc: 0.5240 - val_loss: 0.6681 - val_acc: 0.7673
Epoch 2/25
50000/50000 [==============================] - 96s - loss: 0.5468 - acc: 0.8090 - val_loss: 0.4720 - val_acc: 0.8416
Epoch 3/25
50000/50000 [==============================] - 96s - loss: 0.4344 - acc: 0.8487 - val_loss: 0.4472 - val_acc: 0.8477
Epoch 4/25
50000/50000 [==============================] - 96s - loss: 0.3938 - acc: 0.8653 - val_loss: 0.3992 - val_acc: 0.8585
Epoch 5/25
50000/50000 [==============================] - 96s - loss: 0.3672 - acc: 0.8733 - val_loss: 0.3979 - val_acc: 0.8663
Epoch 6/25
50000/50000 [==============================] - 96s - loss: 0.3464 - acc: 0.8809 - val_loss: 0.3717 - val_acc: 0.8771
Epoch 7/25
50000/50000 [==============================] - 96s - loss: 0.3323 - acc: 0.8869 - val_loss: 0.3880 - val_acc: 0.8750
Epoch 8/25
50000/50000 [==============================

In [34]:
for ii in accuracy_scores:
    print ii, accuracy_scores[ii]

linear_models {'linear_SVM': 0.80220000000000002}
cnn_on_fashion_mnist 0.902099995852
custom_on_fashion_mnist 0.873099998832
